# Part 0

## 0.A Import


In [27]:

import numpy  as np
import pandas as pd
from   scipy.interpolate import interp1d
from   scipy.interpolate import interp2d
from   scipy.misc import derivative
from   scipy import optimize
import astropy.units as u
#import astropy.constants as c
from   astropy.cosmology import FlatLambdaCDM, z_at_value
from   tqdm import *
from   sympy import *
from   astropy.cosmology import Planck13 as cosmo
from   astropy import constants as const
import sys
from   scipy.interpolate import interp1d
from   scipy.interpolate import interp2d
from   scipy.special import zeta
import pickle

import matplotlib.pyplot as plt
from   matplotlib import ticker
from matplotlib import gridspec
import matplotlib.pylab as pylab
#from matplotlib import colormaps
import matplotlib.ticker as mticker

sys.path.insert(1, '../packages')
import units
import time

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

#suppress warnings
import warnings
warnings.filterwarnings('ignore')


# Plot parameters
from plot_params import params

pylab.rcParams.update(params)

cols_default = plt.rcParams['axes.prop_cycle'].by_key()['color']


## 0.B General Constants and Functions 

Hydrogen number density

\begin{equation}
n_\text{H} = (1-Y_p) n_b = (1-Y_p) \times \eta \frac{2 \zeta(3)}{\pi^2} T^3_\gamma,
\end{equation}

Proton number density (proton from both Hydrogen and Helium)

\begin{equation}
n_p = \left( 1 - \frac{Y_p}{2} \right) n_b = \left( 1 - \frac{Y_p}{2} \right) \times \eta \frac{2 \zeta(3)}{\pi^2} T^3. 
\end{equation}

In [28]:

# Fundamental Constants
h = 4.135667696*1e-15          # in eV s

hbar = h / (2*np.pi)           # in eV s

c = 2.99792458*1e10            # in cm/s

mpl = 1.2209 * 10**19 * 10**9  # in eV

# Defined by Xucheng

# Consistent with my MMA
cm_to_m     = 10**(-2)
cm_to_km    = 10**(-5)

m_to_cm     = 1/cm_to_m
km_to_cm    = 1/cm_to_km

pc_to_cm    = 3.08567758149137* 10**18
Mpc_to_cm   = 10**6 * pc_to_cm

cmInv_to_eV = hbar*c
sInv_to_eV  = hbar
eV_to_J     = 1.602176634 * 10**(-19)
kB_to_JperK = 1.38064852 * 10**(-23)

eV_to_K = eV_to_J/kB_to_JperK

J = 1/eV_to_J  # in eV
K = 1/eV_to_K  # in eV

# Cosmological Constants

# Planck 2018 paper VI Table 2 Final column (68% confidence interval)
planck18_cosmology = {'Oc0': 0.2607,
                              'Ob0': 0.04897,
                              'Om0': 0.3111,
                              'Hubble0': 67.66,
                              'n': 0.9665,
                              'sigma8': 0.8102,
                              'tau': 0.0561,
                              'z_reion': 7.82,
                              't0': 13.787,
                              'Tcmb0': 2.7255,
                              'Neff': 3.046,
                              'm_nu': [0., 0., 0.06],
                              'z_recomb': 1089.80,
                              'reference': "Planck 2018 results. VI. Cosmological Parameters, "
                                           "A&A, submitted, Table 2 (TT, TE, EE + lowE + lensing + BAO)"
                              }


# TCMB_0 from Planck 2018
TCMB_0   = planck18_cosmology["Tcmb0"]*K # in eV

# H = h_Hubble * 100 km/s/Mpc
# Note: 'h' is already used for Planck constant before
h_Hubble = planck18_cosmology["Hubble0"]/100

H0       = h_Hubble * ( 100 * km_to_cm * sInv_to_eV/ Mpc_to_cm )   # in eV

# Faction of matter
Omega_m  = planck18_cosmology["Om0"]

# Fraction of radiation
Omega_r = (8*np.pi**3/90) * 3.38 * (TCMB_0**4/(mpl**2 * H0**2))

# Fraction of dark energy
Omega_Lambda = 1 - Omega_r - Omega_m

# Baryon-to-photon ratio, from 1912.01132
eta = 6.129 * 10**(-10)

# Helium-to-hydrogen mass fraction, from 1912.01132
Yp  = 0.247

# Electron mass (from PDG)
m_e = 0.511 * 10**6 # in eV

# From PDG
alpha = 1/137.035999084

ee = np.sqrt(4*np.pi*alpha)

# Thompson Scattering Cross Section (Double-checked with Wiki)
thomson_xsec = 6.6524587158e-25   # cm^2

# electron’s Compton wavelength (Double-checked with Wiki)
# lambda_e = 2.426e-10  cm
lambda_e = (2*np.pi/m_e)*cmInv_to_eV # in cm

# red shift for mu-y transition era [z_trans_1, z_trans_2]
z_trans_1 = 10**4
z_trans_2 = 3 * 10**5

# Hubble Parameter
# in eV
def hubble(z):
    
    return H0 * np.sqrt(Omega_Lambda + Omega_m * (1.0 + z) ** 3 + Omega_r * (1.0 + z) ** 4)

# n_p function [cm^-3]
def n_p(z):
    
    n_p_0 = (1-Yp/2) * eta * (2*zeta(3)/np.pi**2) * ( TCMB_0 / cmInv_to_eV )**3   # in cm^-3
    
    n_p_z = n_p_0 * (1+z)**3   # in cm^-3
    
    return n_p_z


# n_H function [cm^-3]
def n_H(z):
    
    n_H_0 = (1-Yp) * eta * (2*zeta(3)/np.pi**2) * ( TCMB_0 / cmInv_to_eV )**3   # in cm^-3
    
    n_H_z = n_H_0 * (1+z)**3   # in cm^-3
    
    return n_H_z



# ========================================================
# x_e function defined by Andrea
# ========================================================

x_e_data = pickle.load(open("../data/std_soln_He.p", "rb"))

def x_e(z):
    
    return np.interp(z, np.flipud(x_e_data[0]) - 1.0, np.flipud(x_e_data[2]))

# Part 1

## 1.A.0 Exact Calculation for $X_e$

Free electron fraction (total)

$X_e = \frac{n_{e}}{n_\text{H}} = \frac{ n_{e,\text{H}} + n_{e,\text{He}} }{ n_\text{H} }$ 

Free electron fraction (from Hydrogen)

$X_{e,\text{H}} = \frac{n_{e,\text{H}}}{n_\text{H}}$

In [30]:
# Parameters used in Class
class_parameters = {'H0':         planck18_cosmology["Hubble0"],
                    'Omega_b':    planck18_cosmology["Ob0"],
                    'N_ur':       planck18_cosmology["Neff"],
                    'Omega_cdm':  planck18_cosmology["Oc0"],
                    'YHe':        Yp,
                    'z_reio':     planck18_cosmology["z_reion"]}

# Call class
from classy import Class

CLASS_inst = Class()
CLASS_inst.set(class_parameters)
CLASS_inst.compute()

# z array
z_ary_log     = np.logspace(-6, 8, 100000)



# Add 0(today) at the beginning of z_ary
z_ary = np.insert(z_ary_log,0,[0])

# Xe array
Xe_ary   = np.array([ CLASS_inst.ionization_fraction(z) for z in z_ary ])

# XeH array
# (If Xe>=1, replace it with 1)
XeH_ary = np.where(Xe_ary<=1,Xe_ary,1)

# Xe interpolation
Xe_interp  = interp1d(z_ary, Xe_ary, fill_value="extrapolate")

# XeH interpolation
XeH_interp = interp1d(z_ary, XeH_ary, fill_value="extrapolate")

# dXe/dz array
# (with the length of z_ary[:-1])
dXe_dz_ary = np.diff(Xe_ary)/np.diff(z_ary)

# dXeH/dz array
# (with the length of z_ary[:-1])
dXeH_dz_ary = np.diff(XeH_ary)/np.diff(z_ary)


# dXe/dz interpolation
dXe_dz_interp  = interp1d(z_ary[:-1], dXe_dz_ary, fill_value="extrapolate")

# # dXeH/dz interpolation
dXeH_dz_interp  = interp1d(z_ary[:-1],  dXeH_dz_ary, fill_value="extrapolate")



# n_e function [cm^-3]
def n_e(z):
    
    n_e_z = Xe_interp(z) * n_H(z)
    
    return n_e_z   # in cm^-3


ne0 = n_e(0)

# mA^2 at z=0
mASq0 = ee**2 * (ne0 * cmInv_to_eV**3)/m_e  # in eV^2

## 1.A.1 Special Constants and Functions 

$\alpha_\rho= \frac{\zeta(3)}{3 \zeta(4)} \simeq 0.3702$

$\alpha_\mu = \frac{\zeta(2)}{3\zeta(3)}  \simeq 0.456$

$x_0 = \frac{4}{3 \alpha_\rho} = \frac{4\zeta(4)}{\zeta(3)} \simeq 3.6$

$\kappa_c = \frac{45}{\pi^4} \left( \frac{2 \pi^6}{ 135 \zeta(3) } - 6 \zeta(3) \right) \simeq 2.14185$

$\rho_\gamma(T) = \frac{g_\gamma \pi^2}{30} T^4 = \frac{\pi^2}{15} T^4$


In [45]:
alpha_rho = zeta(3)/(3*zeta(4))
alpha_mu  = zeta(2)/(3*zeta(3))
x_0 = (4*zeta(4))/zeta(3)
kappa_c = (45/np.pi**4)*( 2*np.pi**6/(135*zeta(3)) - 6*zeta(3) )

def rho_gamma(T):
    
    return (np.pi**2/15) * T**4

## 1.A.2 $m_\gamma^2(z,\omega_0)$, $d m_\gamma^2/dz(z,\omega_0)$, and $z_\text{cross}$

From arXiv:2002.05165, we have
\begin{equation}
m_\gamma^2(z) = 1.4 \times 10^{-21}\text{eV}^2 X_e(z) \left(\frac{n_\text{H}(z)}{\text{cm}^{-3}}\right)- 8.4 \times 10^{-24} \text{eV}^2 \left(\frac{\omega(z)}{\text{eV}}\right)^2 \left( 1 - X_{e,\text{H}}(z) \right) \left(\frac{ n_\text{H}(z) }{ \text{cm}^{-3} }\right),
\end{equation}

where (Here $n_{e} = n_{e,\text{H}} + n_{e,\text{HI}}$)
\begin{equation}
X_e = \frac{n_{e}}{n_\text{H}} \quad \text{and} \quad X_{e,\text{H}} = \frac{n_{e,\text{H}}}{n_\text{H}},
\end{equation}

and 
\begin{equation}
n_H = (1-Y_p) n_\gamma = (1-Y_p) \times \eta \frac{2 \zeta(3)}{\pi^2} T^3.  
\end{equation}


Doing the derivative over $z$, we have
\begin{equation}
\begin{aligned}
\frac{dm_\gamma^2(z)}{dz} 
& = 1.4 \times 10^{-21}\text{eV}^2 \left[ \frac{dX_e(z)}{dz} + \frac{3 X_e(z)}{1+z} \right] \left(\frac{n_\text{H}(z)}{\text{cm}^{-3}}\right)\\
& \quad - 8.4 \times 10^{-24} \text{eV}^2 \left(\frac{\omega(z)}{\text{eV}}\right)^2 \left[ - \frac{dX_{e,\text{H}}(z)}{dz} + \frac{5\left( 1 - X_{e,\text{H}}(z) \right)}{1+z} \right] \left(\frac{ n_\text{H}(z) }{ \text{cm}^{-3} }\right),
\end{aligned}
\end{equation}

In [46]:
# Plasma mass^2 [eV^2]
# Input omega0 in eV
def mAsq(z, omega0):
    
    # from free electron
    mAsq_1 = 1.4*10**(-21) * Xe_interp(z) * n_H(z)
    
    # from neutral hydrogen
    mAsq_2 = - 8.4 * 10**(-24) * ( omega0 * (1+z) )**2 * (1-XeH_interp(z) ) * n_H(z)
    
    # total mA^2
    mAsq = mAsq_1 + mAsq_2
    
    return mAsq


# d(Plasmon mass^2)/dz [eV^2]
# Input omega0 in eV
def dmAsq_over_dz(z, omega0):
    
    # from free electron
    dmAsq_over_dz_1 = 1.4*10**(-21)*( dXe_dz_interp(z) + 3*Xe_interp(z)/(1+z) ) * n_H(z)
    
    # from neutral hydrogen
    dmAsq_over_dz_2 = - 8.4 * 10**(-24) * ( omega0 * (1+z) )**2 * ( -dXeH_dz_interp(z) + 5*(1-XeH_interp(z))/(1+z) ) * n_H(z)
    
    # total d(mA^2)/dz
    dmAsq_over_dz = dmAsq_over_dz_1 + dmAsq_over_dz_2
    
    return dmAsq_over_dz


# Get z at A-Ap crossings
# Modified from 'grf.py' file in https://github.com/smsharma/dark-photons-perturbations

# z_ary: Grids to check whether the level crossing exists
# z_ary = np.logspace(-5, 8, 20000)
# np.logspace(-5, 8, 2000)
def get_z_crossings(mAp, omega0, z_ary = np.logspace(-3, 8, 10000)):
    
    # mA^2 on grids
    mAsq_ary = mAsq(z_ary, omega0)
    
    # Find the grid containing level crossing
    # np.where is used to find the position of 'True' in array
    # [:-1] means 1,2,3,...,N-1, [1:] means 2,3,...,N-1,N
    where_ary =  np.where( np.logical_or( 
            (mAsq_ary[:-1]<mAp**2) * (mAsq_ary[1:]>mAp**2), (mAsq_ary[:-1]>mAp**2) * (mAsq_ary[1:]<mAp**2) ) )
    
    # mA^2-mAp^2 on grids
    def mAsq_minus_mApsq_ary(z):
        
        return mAsq(z, omega0) - mAp**2
    
    # We use list, because '.append' can be applied in list, but not array
    z_cross_list = []
    
    # Solve z_cross for each grid containing level crossing
    for i in range(len(where_ary[0])):
        z_cross_list.append(
            optimize.brenth(mAsq_minus_mApsq_ary, z_ary[where_ary[0][i]], z_ary[where_ary[0][i] + 1] ) )
#         z_cross_list.append(
#             optimize.brentq(mAsq_minus_mApsq_ary, z_ary[where_ary[0][i]], z_ary[where_ary[0][i] + 1] ) )

    
    # Make list to array
    z_cross_ary = np.array(z_cross_list)
    
    return z_cross_ary

# Part 1.A.3
## $P_{\gamma \rightarrow A'}$

Perturbative: 
\begin{equation}
P_{\gamma \rightarrow A'}(m_{A'},x) = \sum_\text{res} \frac{\pi \epsilon^2 m_{A'}^4}{ \omega(z_\text{res}) \cdot (1+z_\text{res}) H_\text{res} } \frac{1}{\left| d m_\gamma^2/dz \right|_\text{res}}.
\end{equation}

Non-perturbative: 
\begin{equation}
P_{\gamma \rightarrow A'} = 1 - \exp\left(- \sum_\text{res} \frac{\pi \cdot \sin^2\epsilon \cdot m_{A'}^4}{ \omega(z_\text{res}) \cdot (1+z_\text{res}) H_\text{res} } \frac{1}{\left| d m_\gamma^2/dz \right|_\text{res}} \right)
\end{equation}

In [47]:
# Perturbative P(A->Ap): Needs to input z_res_ary
def P_pre_over_eps2(mAp, x, z_res_ary):
    
    # omega at z=0
    omega0 = x * TCMB_0  # in eV
    
    # omega at z_res
    omega_res_ary = omega0 * (1+z_res_ary) # in eV
    
    # P/eps^2 ary
    P_pre_over_eps2_ary = (np.pi * mAp**4)/( omega_res_ary * (1+z_res_ary) * hubble(z_res_ary) ) * ( 1 / np.abs(dmAsq_over_dz(z_res_ary, omega0)) ) 
    

    
    # P/eps^2
    P_pre_over_eps2  = np.sum(P_pre_over_eps2_ary)
    
    return P_pre_over_eps2


# Perturbative P(A->Ap):  z_res_ary is solved inside
def P_over_eps2(mAp, x):
    
    # omega at z=0
    omega0 = x * TCMB_0  # in eV
    
    # z_res array
    z_res_ary = get_z_crossings(mAp, omega0)
    
    # P_tot
    P_over_eps2 = P_pre_over_eps2(mAp, x, z_res_ary)
    
    return P_over_eps2



In [48]:
# ----------------------------------------------------------------------
# Parameters for scanning in (mAp,x) plane
# Here we calculate P(A->Ap) for (mAp,x) plane. Result will be used later
# To interpolate probability for much faster calculations. 
# We have checked and error between interpolated and directly calcualted probability is negiligible.
# ----------------------------------------------------------------------
# Output the P(A->Ap) table on x-mAp plane
N_P_x   = 500
N_P_mAp = 300

# N_P_x   = 1000
# N_P_mAp = 300

nu_P_min = 10**(-2)  # in cm^-1
nu_P_max = 30        # in cm^-1    # Note: The highest nu_FIRAS is 21.33 cm^-1

x_P_min = 2 * np.pi * nu_P_min * cmInv_to_eV/TCMB_0   # in eV
x_P_max = 2 * np.pi * nu_P_max * cmInv_to_eV/TCMB_0   # in eV

mAp_P_min = 10**(-17)                               # in eV     # Note: The lower mAp of FIRAS bound is ~10^(-15) eV
mAp_P_max = 10**(-3)                                # in eV     # Note: mAp~10^-4eV, z_res~10^6 (Double Compton Scattering). 
# mAp_P_max = 0.3*m_Aprime_res_RAD(z_ary_log[-1])   # in eV     # Note: mAp~10^-4eV, z_res~10^6 (Double Compton Scattering). 

# mAp_P_min = 10**(-11)                             # in eV     # Note: The lower mAp of FIRAS bound is ~10^(-15) eV
# mAp_P_max = 10**(-3)                              # in eV     # Note: mAp~10^-4eV, z_res~10^6 (Double Compton Scattering). 

print('N_P_x     = ', N_P_x )
print('N_P_mAp   = ', N_P_mAp )
print('')
print('x_P_min   = ', x_P_min )
print('x_P_max   = ', x_P_max )
print('')
print('mAp_P_min = ', mAp_P_min, 'eV')
print('mAp_P_max = ', mAp_P_max, 'eV')
print('')

# ----------------------------------------------------------------------
# Scan in (x,mAp) plane
# ----------------------------------------------------------------------

x_P_scan_ary   = np.linspace( x_P_min, x_P_max, N_P_x )
mAp_P_scan_ary = np.logspace( np.log10(mAp_P_min), np.log10(mAp_P_max), N_P_mAp )

P_over_eps2_scan_2Dary = np.zeros( (len(x_P_scan_ary),len(mAp_P_scan_ary)) )

zres_scan_2Dary        = np.zeros( (len(x_P_scan_ary),len(mAp_P_scan_ary)) )

N_P_x     =  500
N_P_mAp   =  300

x_P_min   =  0.0052789483668742995
x_P_max   =  15.836845100622897

mAp_P_min =  1e-17 eV
mAp_P_max =  0.001 eV



In [37]:


for j in tqdm_notebook(range(0,len(mAp_P_scan_ary))):

    
    for i in range(0,len(x_P_scan_ary)):
        
        x_i   = x_P_scan_ary[i]
        mAp_j = mAp_P_scan_ary[j]   # in eV
        
        get_z_crossing_ij = get_z_crossings(mAp_j,TCMB_0*x_i)
        
        # Check whether A and Ap crosses
        # If no crossing, choose zres=0
        if (len( get_z_crossing_ij ) > 0):
            zres_scan_2Dary[i][j] = get_z_crossing_ij[0]   # Only pick the smallest z_cross
        else:
            zres_scan_2Dary[i][j] = 0  # When there is no crossing
        
        # P_over_eps2_scan_2Dary[i][j] = P_over_eps2(mAp_j, x_i)
        P_over_eps2_scan_2Dary[i][j] = P_pre_over_eps2(mAp_j, x_i, get_z_crossing_ij)

In [38]:
# Exporting Probability and mAp and x_p values
np.savez("../data/Probability.npz",
         x_P_scan_ary=x_P_scan_ary,
         mAp_P_scan_ary=mAp_P_scan_ary,
         P_over_eps2_scan_2Dary=P_over_eps2_scan_2Dary,
         zres_scan_2Dary=zres_scan_2Dary
         )

In [39]:

# # **********************************************************************
# # Import the result
# # **********************************************************************

# # -------------------------------------------------------


file_name = "../data/Probability.npz"
if file_name is not None:
    file = np.load(file_name)
# x:   1D array
x_1Dary_P_import = file['x_P_scan_ary']
# -------------------------------------------------------
# mAp: 1D array
mAp_1Dary_P_import = file['mAp_P_scan_ary']

# -------------------------------------------------------
# P/eps^2: 2D array
P_over_eps2_2Dary_import = file['P_over_eps2_scan_2Dary']


zres_2Dary_import=file['zres_scan_2Dary']


In [49]:
# Interpolation: Log10(P/eps^2), Log10(z_res)
# Input: Log10(mAp), x

Reg_trans= 10**(-100)

log10P_over_eps2_interp = interp2d( np.log10(mAp_1Dary_P_import), x_1Dary_P_import, np.log10(P_over_eps2_2Dary_import + Reg_trans) )
log10zres_interp        = interp2d( np.log10(mAp_1Dary_P_import), x_1Dary_P_import, np.log10(zres_2Dary_import + Reg_trans)  )

# P/eps^2 from 2D interpolation
# Input: Log10(mAp), x
def P_over_eps2_interp(mAp, x):
    
    P_over_eps2_interp = 10**log10P_over_eps2_interp( np.log10(mAp), x )
    
    return P_over_eps2_interp

# zres from 2D interpolation
# Input: Log10(mAp), x
# Note! Here, 'zres_interp' only picks the minimal zres when there is multi solution
def zres_interp(mAp,x):
    
    zres_interp = 10**log10zres_interp( np.log10(mAp), x )
    
    return zres_interp



## 1.B.0 Functions (Special, for $\mu$-era)

Blackbody Intensity

$I^{(0)}(x) = \frac{2T_0^3}{h^2 c^2} \frac{x^3 e^x}{e^x-1} = \frac{T^3_0}{2\pi^2} \frac{x^3 e^x}{e^x-1}$

Temperature shift

$\mathcal{T}(x) = \frac{x e^x}{e^x-1}$

$G^{\text{Chluba}}(x) = I^{(0)}(x) \cdot \mathcal{T}(x)$ 


Units

$1 \, h = 6.626 \times 10^{-34} J \cdot s = 4.136 \times 10^{-15} \text{eV} \cdot s$

$1 \,\text{eV} = 1.602 \times 10^{-19}\,J$

$1 \,J = 1 \,\text{kg}\cdot m^2 \cdot s^{-2}$

$1 \,\text{Jy} = 10^{26} \,\text{kg} \cdot s^{-2}$

In [50]:
def J_bb(z):
    
    z_mu = 1.98e6
    
    return 0.983 * np.exp(-(z/z_mu)**2.5) * (1 - 0.0381 * (z/z_mu)**2.29)


# Surviving Probability
# From Chluba 2015 1506.06582
def survival_prob(x, z):
    
    xc_DC = 8.60e-3 * ((1.+z)/2e6)**0.5
    xc_BR = 1.23e-3 * ((1.+z)/2e6)**(-0.672)
    
    xc = np.sqrt(xc_DC**2 + xc_BR**2)
    
    return np.exp(-xc/x)


def lambda_func(x_prime, z_prime):
    
    return alpha_rho * (x_prime - (x_prime - x_0 * survival_prob(x_prime, z_prime)) * J_bb(z_prime))


# Temperature Shift
def T_shift(x):
    
    T_shift = np.where(x<=300, x*np.exp(x)/(np.exp(x) - 1), x)
    
    return T_shift




# Black Body Intensity
def I0(x, T0, units='eV_per_cmSq'):
    
    if   units == 'eV_per_cmSq': # units:    eV * cm^-2 * sr^-1
        prefac = 1
    elif units == 'SI':          # SI units: kg * s^-2 * sr^-1
        prefac = eV_to_J * (1/cm_to_m)**2
    elif units == 'MJy':         # units:    MJy * sr^-1
        prefac = eV_to_J * (1/cm_to_m)**2 * 1e26 / 1e6
        
    I0 = np.where(x<=300, prefac * 2 * T0**3 / h**2 / c**2  * x**3 / (np.exp(x)-1), 0 )
        
    return I0


# rho_bar(x):
# Normalized and Dimless Intensity, Defined in Hook2023
def rho_bar(x):
    
    return (15/np.pi**4) * x**3/(np.exp(x)-1)


# G-function (Chluba 15)
def G(x, T0, units='eV_per_cmSq'):

    return I0(x, T0, units=units) * T_shift(x)


# M-function (Chluba 15)
def M(x, T0, units='eV_per_cmSq'):
    
    return G(x, T0, units=units) * (alpha_mu - 1./x)



## 1.B.1 Functions (Special, for y-era)

Compton y-parameter for photon distribution

\begin{equation}
y_\gamma = \int \frac{T_\gamma}{m_e} \frac{\sigma_T n_e}{H(z)(1+z)} dz.
\end{equation}


Compton y-parameter for electron distribution

\begin{equation}
y_e = \int \frac{T_e}{m_e} \frac{\sigma_T n_e}{H(z)(1+z)} dz.
\end{equation}

In the definition of $y_\gamma$ and $y_e$, we fix $T_0=2.7255\text{K}$. 


BR emissivity

\begin{equation}
\Lambda_\text{BR} = \frac{\alpha \lambda_e^3}{2 \pi \sqrt{6 \pi}} n_{\text{free} p} \theta_e^{-7/2} g_\text{ff}(x_e),
\end{equation}

where $\theta_e = T_e/m_e$, $x_e = \omega/T_e$, $n_{\text{free} p} = X_{e,\text{H}} n_\text{H}$. 

Photon survival probability

\begin{equation}
P_s(x,z) \simeq e^{-\tau_\text{ff}(x,z)},
\end{equation}
where 

\begin{equation}
\tau_\text{ff}(x,z) = \int^z_0 \frac{\Lambda_\text{BR}(z,x_e) ( 1 - e^{-x_e} )}{x_e^3} \frac{\sigma_T n_e c dz}{H(1+z)}
\end{equation}

In [51]:
# Y-function (Chluba 15)
def Y(x, T0, units='eV_per_cmSq'):
    
    Y = G(x, T0, units=units) * ( x/np.tanh(x/2)-4 )
    
    return Y

def f(x):
    
    return np.exp(-x) * ( 1 + x**2/2 )

# Compton-y parameter (Photon)
# Modified by Xucheng by replacing ``x_e(z_to_int) * ne0 * (1.0 + z_to_int) ** 3'' with ``n_e(z)''
def y_gamma(z):
    
    z_to_int = np.logspace(-5, np.log10(z), 500)
    
    fac_1 = TCMB_0 * (1.0 + z_to_int) / m_e

    fac_2 = thomson_xsec * n_e(z_to_int) * c / ( hubble(z_to_int) / hbar * (1.0 + z_to_int))

    return np.trapz(fac_1 * fac_2, z_to_int)


# T_e: Electron Temperature
# in eV   
def T_e(z):
    
    try:
        
        _ = iter(z)
        
        T_out = np.zeros_like(z)

        T_out[z  < 2999] = np.interp(z[z < 2999], np.flipud(x_e_data[0]) - 1.0, np.flipud(x_e_data[1]))
        T_out[z >= 2999] = (1.0 + z[z >= 2999]) * TCMB_0
    
    except:
        
        if z < 2999:
            
            T_out = np.interp(z, np.flipud(x_e_data[0]) - 1.0, np.flipud(x_e_data[1]))

        else:
            
            T_out = (1.0 + z) * TCMB_0

    return T_out


# Approx gff for high x (Draine)
def approx_high_x_Draine(x, T_e):
    
    theta_e = T_e / m_e
        
    return np.log(np.exp(1.0) + np.exp(5.960 - np.sqrt(3) / np.pi * np.log(270.783 * x * theta_e ** (-0.5))))


# Approx gff for low x (Draine)
def approx_low_x_Draine(x, T_e):
    
    theta_e = T_e / m_e
    
    return 4.691 * (1.0 - 0.118 * np.log(27.0783 * x * theta_e ** (-0.5)))


# Gaunt factor: g_ff    (Modifed by X.G)
# x    : 1D array N_x
# g_ff : 1D array N_x
def g_ff(x, T_e):  # here we define the Gaunt factor as in https://astrojacobli.github.io/astro-ph/ISM/Draine.pdf, sec.10.2
    
    # thr = 1e-3
    
    g_ff_out = approx_high_x_Draine(x, T_e)

    return g_ff_out


# Lambda_BR           (Modifed by X.G)
# x    : 1D array N_x
# XeH  : 1D array N_XeH

# [[[Warning: Xe is the ionization ratio, not the dimless frequency]]]
def Lambda_BR(x, XeH, z):
    
    theta_e = T_e(z) / m_e
    
    # Number density of free proton
    # 1D array N_XeH
    # n_p = ne0 * (1.0 + z) ** 3 * XeH  # in cm^-3
    n_free_p = n_H(z) * XeH  # in cm^-3

    # 2D array N_x * 1
    x_electron = x[:,None] * TCMB_0 * (1.0 + z) / T_e(z)

    # 2D array N_x * N_Xe
    Lambda_BR = alpha * lambda_e ** 3 / (2 * np.pi * np.sqrt(6 * np.pi)) * n_free_p * theta_e ** (-7.0 / 2.0) * g_ff(x_electron, T_e(z))
    
    return Lambda_BR


# tau_ff
# x    : 1D array N_x
# z    : Number
def tau_ff(x, z):
    
    N_z_to_int = 500
    
    # 1D array N_z_to_int:  small number to z
    z_to_int = np.logspace(-5, np.log10(z), N_z_to_int)
    
    # 2D array N_x * N_z_to_int
    # [:, None] make x to transfer from 1D array to N_x * 1 2D array
    x_electron = x[:,None] * TCMB_0 * (1.0 + z_to_int) / T_e(z_to_int)

    # 2D array N_x * N_z_to_int
    # <<< XG replace 'x_e' with 'XeH_interp' >>>
    Lambda = Lambda_BR(x, XeH_interp(z_to_int), z_to_int)

    # 2D array N_x * N_z_to_int
    fac_1 = Lambda * (1 - np.exp(-x_electron)) / x_electron ** 3

    # 1D array N_z_to_int
    # <<< XG replace 'x_e' with 'XeH_interp' >>>
    # <<< XG replace 'ne0 * (1.0 + z_to_int) ** 3' with 'n_e(z)' >>>
    fac_2 = thomson_xsec * n_e(z_to_int) * c / (hubble(z_to_int) / hbar * (1.0 + z_to_int))
    
    # 1D array N_x
    # ( Note that fac_1 * fac_2 is a 2D array N_x * N_z_to_int )
    tau_ff = np.trapz(fac_1 * fac_2, z_to_int)
    
    return tau_ff

# 4. Green's Function

## 4A. Green's Function ($\mu$-era)

Green's function for $\mu$-era

\begin{equation}
G^\text{Chluba}_\text{$\mu$-era}(x,x',z';\color{magenta}{T_0}) = G^\text{Chluba}_{\mu}(x,x',z';\color{magenta}{T_0}) + G^\text{Chluba}_{\mathcal{T}}(x,x',z';\color{magenta}{T_0}). 
\end{equation}

M-Term ($3/\kappa_c \simeq 1.4$)

\begin{equation}
G^\text{Chluba}_{\mu}(x,x',z';\color{magenta}{T_0}) = \frac{3 \alpha_\rho x'}{\kappa^c} \left( 1 - P_s(x',z') \frac{x_0}{x'} \right) \cdot J^*(z') \cdot M^\text{Chluba}(x;\color{magenta}{T_0})
\end{equation}

T-Term

\begin{equation}
G^\text{Chluba}_{ \mathcal{T} }(x,x',z';\color{magenta}{T_0}) = \frac{1}{4} \lambda(x',z') \cdot G^\text{Chluba}(x;\color{magenta}{T_0}),
\end{equation}

where 

\begin{equation}
\lambda(x',z') = \alpha_\rho x' \left[ 1 - \left( 1 - P_s(x',z') \frac{x_0}{x'} \right) J^*(z') \right]. 
\end{equation}

In [ ]:
# # Green's Function in mu-era: M-part
# def greens_mu_M(x, x_prime, z_prime,  T0, units='eV_per_cmSq'):
    
#     # 2D array  N_x * N_xp
#     # Note that:
#     # f(x)            : 1D array N_x
#     # f(x)[:,None]    : 2D array N_x * N_xp
#     greens_mu_ary = (   3 * alpha_rho / kappa_c
#         * (x_prime - x_0 * survival_prob(x_prime, z_prime))
#         * J_bb(z_prime) * M(x, T0, units = units)[:, None]   )
    
#     return greens_mu_ary

# # Green's Function in mu-era: T-part
# def greens_mu_T(x, x_prime, z_prime,  T0, units='eV_per_cmSq'):
    
#     # 2D array  N_x * N_xp
#     # Note that:
#     # f(x)            : 1D array N_x
#     # f(x)[:,None]    : 2D array N_x * N_xp
#     greens_mu_ary = ( lambda_func(x_prime, z_prime) * G(x, T0, units = units)[:, None] / 4. )
    
#     return greens_mu_ary

# # Green's Function in mu-era: M+T
# def greens_mu_MT(x, x_prime, z_prime,  T0, units='eV_per_cmSq'):
    
#     # 2D array  N_x * N_xp
    
#     greens_mu_M_ary =  greens_mu_M(x, x_prime, z_prime, T0, units=units)
    
#     greens_mu_T_ary =  greens_mu_T(x, x_prime, z_prime, T0, units=units)
    
#     greens_mu_ary   =  greens_mu_M_ary + greens_mu_T_ary
    
#     return greens_mu_ary

## 4B. Green's Function (y era)

### $G_y$: Green's function for y-era

\begin{equation}
\begin{aligned}
G_y^\text{Chluba}(x,x',z';\color{magenta}{T_0}) = G_Y^\text{Chluba}(x,x',z';\color{magenta}{T_0}) + G_\text{Doppler}^\text{Chluba}(x,x',z';\color{magenta}{T_0}),
\end{aligned}
\end{equation}

where

\begin{equation}
\boxed{ G_Y^\text{Chluba}(x,x',z';\color{magenta}{T_0}) = \alpha_\rho x' \left(1 - \frac{e^{ 4 (\alpha+\beta) y_\gamma(z') } e^{-\tau_\text{ff}(x',z')} }{1+x' y_\gamma(z')}\right) \frac{Y^\text{Chluba}(x, \color{magenta}{T_0})}{4} }
\end{equation}

and

\begin{equation}
\boxed{ G_\text{Doppler}^\text{Chluba}(x,x',z';\color{magenta}{T_0}) = \alpha_\rho x' \frac{\rho_\gamma(\color{magenta}{T_0})}{4\pi} \frac{2\pi}{\color{magenta}{T_0}} \frac{ e^{-\frac{1}{4 \beta y_\gamma(z')} \left\{\ln\left[x \left(1/x' + y_\gamma(z') \right)\right] - \alpha y_\gamma(z') \right\}^2 }  }{ x' \sqrt{4 \pi \beta y_\gamma(z')}} e^{-\tau_\text{ff}(x',z')} }.
\end{equation}

Here, 

\begin{equation}
\rho_\gamma(T_0) = \frac{\pi^2}{15} T_0^4
\end{equation}

and

\begin{equation}
\alpha(x',z') = \frac{3 - 2 f(x')}{ \sqrt{1+x'y_\gamma(z')} }, \quad \quad \beta(x',z') = \frac{1}{1+x' y_\gamma(z') \left[ 1 - f(x') \right] }. 
\end{equation}

In [ ]:
# # alpha function defined in Hook2023 and Chluba2015
# def alpha_func(x_prime,z_prime):
    
#     # Compton y-parameter (photon)
#     y = y_gamma(z_prime)
    
#     # alpha
#     alpha_func = ( 3 - 2 * f(x_prime) )/np.sqrt(1+x_prime * y)
    
#     return alpha_func


# # beta function defined in Hook2023 and Chluba2015
# def beta_func(x_prime,z_prime):
    
#     # Compton y-parameter (photon)
#     y = y_gamma(z_prime)
    
#     # beta
#     beta_func = 1/( 1 + x_prime * y * (1-f(x_prime)) )
    
#     return beta_func


# # Green's function: y-era (Y-part)
# # x   : 1D array N_x
# # xp  : 1D array N_xp
# # T0  : Number  # in eV
# def greens_Y(x,x_prime,z_prime,T0,units="eV_per_cmSq"):
    
#     if   units == 'eV_per_cmSq': # units:    eV * cm^-2 * sr^-1
#         prefac = 1
#     elif units == 'SI':          # SI units: kg * s^-2 * sr^-1
#         prefac = eV_to_J * (1/cm_to_m)**2
#     elif units == 'MJy':         # units:    MJy * sr^-1
#         prefac = eV_to_J * (1/cm_to_m)**2 * 1e26 / 1e6    
    
#     # Compton y-parameter (photon)
#     y = y_gamma(z_prime)
    
#     # <<<Use T0_vary>>> in 'rho_gamma(T0)'
#     # photon energy density
#     rho_gamma = (np.pi**2/15) * T0**4 /( hbar*c )**3  # in eV/cm^3
    
#     # tau_ff
#     # 1D array N_xp
#     tau = tau_ff(x_prime, z_prime)
    
#     # alpha
#     alpha = (3-2 * f(x_prime))/np.sqrt(1+x_prime * y)
    
#     # beta
#     beta = 1/( 1 + x_prime * y * (1-f(x_prime)) )
    
    
#     # <<<Use T0_vary>>> in 'Y(x_prime, T0, units=units)'
#     # 2D array: N_x * N_xp
#     # We use [:,None] to make Y(x) to be a 2D N_x * 1 array
#     # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#     term_Y = (1.0 - np.exp(y * (alpha + beta)) * np.exp(-tau) / (1.0 + x_prime * y)) * Y(x, T0, units="eV_per_cmSq")[:, None] / 4
#     # in eV/cm^2
#     # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
#     # Correct (Modified Chluba)
#     # 2D array: N_x * N_xp
#     green_Y = prefac * term_Y * x_prime * alpha_rho  
    
#     return green_Y


# # Green's function: y-era (Doppler-part)
# # x   : 1D array N_x
# # xp  : 1D array N_xp
# # T0  : Number  # in eV
# def greens_Doppler(x,x_prime,z_prime,T0,units="eV_per_cmSq"):
    
#     if   units == 'eV_per_cmSq': # units:    eV * cm^-2 * sr^-1
#         prefac = 1
#     elif units == 'SI':          # SI units: kg * s^-2 * sr^-1
#         prefac = eV_to_J * (1/cm_to_m)**2
#     elif units == 'MJy':         # units:    MJy * sr^-1
#         prefac = eV_to_J * (1/cm_to_m)**2 * 1e26 / 1e6    
        
#     # Compton y-parameter (photon)
#     y = y_gamma(z_prime)
    
#     # <<<Use T0_vary>>> in 'rho_gamma(T0)'
#     # photon energy density
#     rho_gamma = (np.pi**2/15) * T0**4 /( hbar*c )**3  # in eV/cm^3
    
#     # tau_ff
#     # 1D array N_xp
#     tau = tau_ff(x_prime, z_prime)
    
#     # alpha
#     alpha = (3-2 * f(x_prime))/np.sqrt(1+x_prime * y)
    
#     # beta
#     beta = 1/( 1 + x_prime * y * (1-f(x_prime)) )    
 
#     # 2D array: N_x * N_xp
#     # We use [:,None] to make x to be a 2D N_x * 1 array
#     gaussian = np.exp(-((np.log(x[:, None] / x_prime) - alpha * y + np.log(1 + x_prime * y)) ** 2) / (4 * y * beta)) / (x_prime * np.sqrt(4 * np.pi * y * beta))
    
#     # <<<Use T0_vary>>> in '2pi/T0'
#     # 2D array: N_x * N_xp
#     term_Doppler = c * hbar * ( rho_gamma / (4*np.pi) )  * ( 2*np.pi/T0 ) * np.exp(-tau) * gaussian
#     # in eV/cm^2

#     # Correct (Modified Chluba)
#     # 2D array: N_x * N_xp
#     green_Doppler = prefac * term_Doppler * x_prime * alpha_rho
    
#     return green_Doppler


# # Green's function: y-era (Total)
# # x   : 1D array N_x
# # xp  : 1D array N_xp
# # T0  : Number  # in eV
# def greens_y(x,x_prime,z_prime,T0,units="eV_per_cmSq"):
    
#     if   units == 'eV_per_cmSq': # units:    eV * cm^-2 * sr^-1
#         prefac = 1
#     elif units == 'SI':          # SI units: kg * s^-2 * sr^-1
#         prefac = eV_to_J * (1/cm_to_m)**2
#     elif units == 'MJy':         # units:    MJy * sr^-1
#         prefac = eV_to_J * (1/cm_to_m)**2 * 1e26 / 1e6

#     # Compton y-parameter (photon)
#     y = y_gamma(z_prime)
    
#     # <<<Use T0_vary>>> in 'rho_gamma(T0)'
#     # photon energy density
#     rho_gamma = (np.pi**2/15) * T0**4 /( hbar*c )**3  # in eV/cm^3
    
#     # tau_ff
#     # 1D array N_xp
#     tau = tau_ff(x_prime, z_prime)
    
#     # alpha
#     alpha = (3-2 * f(x_prime))/np.sqrt(1+x_prime * y)
    
#     # beta
#     beta = 1/( 1 + x_prime * y * (1-f(x_prime)) )
    
#     # 2D array: N_x * N_xp
#     # We use [:,None] to make x to be a 2D N_x * 1 array
#     gaussian = np.exp(-((np.log(x[:, None] / x_prime) - alpha * y + np.log(1 + x_prime * y)) ** 2) / (4 * y * beta)) / (x_prime * np.sqrt(4 * np.pi * y * beta))

#     # <<<Use T0_vary>>> in 'Y(x_prime, T0, units=units)'
#     # 2D array: N_x * N_xp
#     # We use [:,None] to make Y(x) to be a 2D N_x * 1 array
#     term_Y = (1.0 - np.exp(y * (alpha + beta)) * np.exp(-tau) / (1.0 + x_prime * y)) * Y(x, T0, units="eV_per_cmSq")[:, None] / 4
#     # in eV/cm^2
    
#     # <<<Use T0_vary>>> in '2pi/T0'
#     # 2D array: N_x * N_xp
#     term_Doppler = c * hbar * ( rho_gamma / (4*np.pi) )  * ( 2*np.pi/T0 ) * np.exp(-tau) * gaussian
#     # in eV/cm^2
    
#     # Correct (Modified Chluba)
#     # 2D array: N_x * N_xp
#     green_y = prefac * (term_Y + term_Doppler) * x_prime * alpha_rho
    
#     return green_y


# # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# # Test of the Code
# # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# x_test          = np.array([10])
# x_prime_test    = np.array([0.1,1,10])
# z_prime_test    = 10**5
# # z_prime_test    = zres_interp(10**(-5),0)[0]
# T0_test         = TCMB_0

# ratio_test = (greens_Y(x_test,x_prime_test,z_prime_test,T0_test,units="eV_per_cmSq") + greens_Doppler(x_test,x_prime_test,z_prime_test,T0_test,units="eV_per_cmSq") )/greens_y(x_test,x_prime_test,z_prime_test,T0_test,units="eV_per_cmSq")

# print('x                     = ', x_test[0])
# print('x_prime               = ', x_prime_test[0])
# print('z_prime               = ', z_prime_test)
# print('T0                    = ', T0_test/K, 'K')
# print('(G_Y+G_Doppler)/G_y   = ', ratio_test[0,0])

# 5. Spectral Distortion in $\mu$-y transition era

## General Formula
Spectral Distortion (begin): 

$$\Delta I(\nu_0; m_{A'}, \epsilon, \color{magenta}{T_0}) = - \frac{1}{2 \zeta(3)} \int dx' dz' \,\, G^{\text{(Chluba)}}(x,x',z'; \color{magenta}{T_0}) \frac{x'^2}{\exp(x')-1} \frac{\Gamma_{\gamma \rightarrow A'}(x',z')}{(1+z')H(z')}$$


$\gamma \rightarrow A'$ Transition Rate: 

$$\Gamma_{\gamma \rightarrow A'}(x',z') = (1+z')H(z') \sum_{\text{res}} P_{\gamma \rightarrow A'} \delta(z'-z_\text{res})$$

$\gamma \rightarrow A'$ Transition Probability (General): 

$$P_{\gamma \rightarrow A'} = \frac{\pi \epsilon^2 m_{A'}^4}{ \omega'(z_\text{res}) \cdot (1+z_\text{res}) H_\text{res} } \frac{1}{\left| d m_\gamma^2/dz \right|_\text{res}}$$

Now we have
$$\Delta I(x; m_{A'}, \epsilon,  \color{magenta}{T_0}) = - \frac{1}{2 \zeta(3)} \int dx' G^{(Chluba)}(x,x',z_\text{res}; \color{magenta}{T_0}) \frac{x'^2}{e^{x'}-1} P_{\gamma \rightarrow A'}(x',z_\text{res})$$

## 5A. Spectral Distortion in $\mu$ era

In [ ]:
# # Delta_I/eps^2 for RAD universe (Approx)
# # x :      1D array N_x
# # xp:      1D array N_xp  (integration variable)
# # output:  1D array N_x

# # ----------------------------------------------------------------------
# # M-Term Only
# # ----------------------------------------------------------------------
# def DeltaI_over_eps2_mu_M_int(x, x_prime_int, m_Aprime, T0, units = 'eV_per_cmSq'):

#     # Here I choose x_prime = 0
#     z_res = zres_interp(m_Aprime, 0)[0]
    
#     # (M-Term Green's Func)
#     # Input x is array:
#     # 2D array: N_x * N_xp  
#     greens_mu_ary = greens_mu_M(x, x_prime_int, z_res, T0, units=units)
    
#     # P(A->Ap)/eps^2
#     # 1D array: N_xp
#     P_over_eps2_ary  = np.transpose(P_over_eps2_interp(m_Aprime, x_prime_int))[0]

#     # Delta_I(A->Ap)
#     # int over xp
#     # 1D array: N_x
#     DeltaI_AToAp_over_eps2 = (-1)/(2*zeta(3)) * np.trapz(greens_mu_ary *  x_prime_int**2/(np.exp(x_prime_int)-1) * P_over_eps2_ary, x_prime_int)
    
#     return DeltaI_AToAp_over_eps2


# # # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# # # Test of the Code
# # # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# # x_test = np.array( [0.1,1,15] )

# # x_prime_int_test = np.logspace(-3,2,1000)

# # # m_Ap_test = np.sqrt( mAsq(5*10**4, 0) )

# # m_Ap_test = 10**(-7) # in eV

# # zres_test = zres_interp(m_Ap_test, 0)[0]  # Here I choose x_prime = 0

# # T0_test   = TCMB_0    #in eV

# # # DeltaI in mu-era: M-term, Integration
# # DeltaI_over_eps2_mu_M_int_test = DeltaI_over_eps2_mu_M_int( x_test, x_prime_int_test, m_Ap_test, T0_test, units = 'MJy')

# # print('--------------------------------------------------')
# # print('For (int)')
# # print('--------------------------------------------------')
# # print('Lower Int Limit of x_prime = ', x_prime_int_test[0])
# # print('Upper Int Limit of x_prime = ', x_prime_int_test[-1])
# # print('Grid Number of x_prime     = ', len(x_prime_int_test))
# # print('')
# # print('============================================================================')
# # print('x                                                         = ', x_test)
# # print('T0                                                        = ', T0_test/K, 'K')
# # print('m_Ap                                                      = ', m_Ap_test, 'eV')
# # print('z_res                                                     = ', zres_test)
# # print('============================================================================')
# # print('Delta_I/eps^2(M)                                          = ', DeltaI_over_eps2_mu_M_int_test, 'MJy')

## 5B. Spectral Distortion in y Era

### $y_\gamma \ll 1$ Limit

Spectral Distortion is

\begin{equation}
\Delta I_y(x, m_{A'}, \epsilon; \color{magenta}{T_0}) = \Delta I_Y(x, m_{A'}, \epsilon; \color{magenta}{T_0}) + \Delta I_\text{Doppler}(x, m_{A'}, \epsilon; \color{magenta}{T_0})
\end{equation}

where

\begin{equation}
\Delta I_Y(x, m_{A'}, \epsilon; \color{magenta}{T_0}) = - \frac{1}{2\zeta(3)} \int dx' G_Y^{\text{(Chluba)}}(x,x',z_\text{res}; \color{magenta}{T_0}) \frac{x'^2}{e^{x'}-1} P_{\gamma \rightarrow A'}(x',z_\text{res})
\end{equation}

and

\begin{equation}
\Delta I_\text{Doppler}(x, m_{A'}, \epsilon; \color{magenta}{T_0}) = - \frac{1}{2\zeta(3)} \int dx' G_\text{Doppler}^{\text{(Chluba)}}(x,x',z_\text{res}; \color{magenta}{T_0}) \frac{x'^2}{e^{x'}-1} P_{\gamma \rightarrow A'}(x',z_\text{res}).
\end{equation}

We already know that

\begin{equation}
G_\text{Doppler}^{\text{(Chluba)}}(x,x',z_\text{res}; \color{magenta}{T_0})  = \alpha_\rho \color{blue}{\frac{\rho_\gamma(T_0)}{4\pi} \frac{2\pi}{T_0} }\frac{ e^{-\frac{1}{4 \beta y_\gamma(z')} \left\{\ln\left[x \left(1/x' + y_\gamma(z') \right)\right] - \alpha y_\gamma(z') \right\}^2 }  }{\sqrt{4 \pi \beta y_\gamma(z')}} e^{-\tau_\text{ff}(x',z')}.
\end{equation}

When $y_\gamma \ll 1$, we have
\begin{equation}
\frac{ e^{-\frac{1}{4 \beta y_\gamma(z')} \left\{\ln\left[x \left(1/x' + y_\gamma(z') \right)\right] - \alpha y_\gamma(z') \right\}^2 }  }{\sqrt{4 \pi \beta y_\gamma(z')}} \simeq x' \delta(x-x'). 
\end{equation}

Therefore, the Doppler smeared distortion can be approximately written as (in $y_\gamma \ll 1$ limit)

\begin{equation}
\Delta I_\text{Doppler}(x, m_{A'}, \epsilon; \color{magenta}{T_0}) = - \frac{\alpha_\rho}{2 \zeta(3)} \left( \frac{\rho_\gamma(T_0)}{4\pi} \frac{2 \pi}{T_0} \right) e^{-\tau_\text{ff}(x,z_\text{res})} \frac{x^3}{e^x-1} P_{\gamma \rightarrow A'}(x,z_\text{res})
\end{equation}


Noting that $\alpha_\rho = \frac{\zeta(3)}{3 \zeta(4)}$, we have

\begin{equation}
\begin{aligned}
\Delta I_\text{Doppler}(x, m_{A'}, \epsilon; \color{magenta}{T_0}) 
& = - \frac{\color{magenta}{T_0^3}}{2 \pi^2} \frac{x^3}{e^x-1} \times P_{\gamma \rightarrow A'}(x,z_\text{res}) e^{-\tau_\text{ff}(x,z_\text{res})} \\
& = - I^{(0)}(x; \color{magenta}{T_0}) \times P_{\gamma \rightarrow A'}(x,z_\text{res}) e^{-\tau_\text{ff}(x,z_\text{res})}
\end{aligned}
\end{equation}

In [ ]:
# # Delta_I/eps^2
# # x :      1D array N_x
# # xp:      1D array N_xp  (integration variable)
# # output:  1D array N_x

# # DeltaI/eps^2: Y-part
# def DeltaI_over_eps2_Y_int(x, x_prime_int, m_Aprime, T0, units = 'eV_per_cmSq'):
    
#     # Here I choose x_prime = 0
#     z_res = zres_interp(m_Aprime, 0)[0]
    
#     # Green's Func at y-era: Y-part
#     # 2D array: N_x * N_xp
#     greens_Y_ary = greens_Y(x, x_prime_int, z_res, T0, units=units)
    
#     # P(A->Ap)/eps^2
#     # 1D array: N_xp
#     P_over_eps2_ary = np.transpose( P_over_eps2_interp(m_Aprime, x_prime_int) )[0]
    
#     # Delta_I(A->Ap)
#     # int over xp
#     # 1D array: N_x
#     DeltaI_over_eps2_Y = (-1)/(2*zeta(3)) * np.trapz(greens_Y_ary *  x_prime_int**2/(np.exp(x_prime_int)-1) * P_over_eps2_ary, x_prime_int)
    
#     return DeltaI_over_eps2_Y



# # Delta_I/eps^2: Doppler-part
# def DeltaI_over_eps2_Doppler_int(x, x_prime_int, m_Aprime, T0, units = 'eV_per_cmSq'):
    
#     # Here I choose x_prime = 0
#     z_res = zres_interp(m_Aprime, 0)[0]
    
#     # Green's Func at y-era: Y-part
#     # 2D array: N_x * N_xp
#     greens_Doppler_ary = greens_Doppler(x, x_prime_int, z_res, T0, units=units)
    
    
#     # P(A->Ap)/eps^2
#     # 1D array: N_xp
#     P_over_eps2_ary = np.transpose( P_over_eps2_interp(m_Aprime, x_prime_int) )[0]
    
#     # Delta_I(A->Ap)
#     # int over xp
#     # 1D array: N_x
#     DeltaI_over_eps2_Doppler = (-1)/(2*zeta(3)) * np.trapz(greens_Doppler_ary *  x_prime_int**2/(np.exp(x_prime_int)-1) * P_over_eps2_ary, x_prime_int)
    
#     return DeltaI_over_eps2_Doppler



# # Delta_I/eps^2: Doppler-part (small y_gamma limit)
# def DeltaI_over_eps2_Doppler_delta(x, m_Aprime, T0, units = 'eV_per_cmSq'):
    
#     if   units == 'eV_per_cmSq': # units:    eV * cm^-2 * sr^-1
#         prefac = 1
#     elif units == 'SI':          # SI units: kg * s^-2 * sr^-1
#         prefac = eV_to_J * (1/cm_to_m)**2
#     elif units == 'MJy':         # units:    MJy * sr^-1
#         prefac = eV_to_J * (1/cm_to_m)**2 * 1e26 / 1e6
    
#     # Here I choose x_prime = 0
#     z_res = zres_interp(m_Aprime, 0)[0]
    
#     # P(A->Ap)/eps^2
#     # 1D array: N_x
#     # Analytically, we integrate out delta(x_prime -x)
#     P_over_eps2_num = np.transpose(P_over_eps2_interp(m_Aprime, x))[0]
    
#     # <<<Use T0_vary>>> in 'rho_gamma(T0)'
#     # photon energy density
#     rho_gamma = (np.pi**2/15) * T0**4  # in eV^4
    
#     # tau_ff
#     # 1D array N_xp
#     # Note: Here, x_prime = x, because of delta(x_prime-x)
#     tau = tau_ff(x, z_res)
    
#     # Delta_I(A->Ap): small y_gamma limit
#     # int over xp
#     # 1D array: N_x
#     # unit: eV^3
#     DeltaI_over_eps2_Doppler_eV3 = - alpha_rho/(2*zeta(3)) * rho_gamma/(4*np.pi) * (2*np.pi/T0) * np.exp(-tau) * x**3/(np.exp(x)-1) * P_over_eps2_num
    
#     # units = units(input)
#     DeltaI_over_eps2_Doppler = prefac * DeltaI_over_eps2_Doppler_eV3 * (1/cmInv_to_eV)**2
    
#     return DeltaI_over_eps2_Doppler



# # Delta_I/eps^2: Doppler-part
# # When y_gamma<=10^-3, switch approximate Doppler term with delta function
# def DeltaI_over_eps2_Doppler_switch(x, x_prime_int, m_Aprime, T0, units = 'eV_per_cmSq'):
    
#     # Here I choose x_prime = 0
#     z_res = zres_interp(m_Aprime, 0)[0]
    
#     # Compton y-parameter (photon)
#     y = y_gamma(z_res)
    
#     # Determine whether do delta-func approximation
#     Approx_Det = (y <= 10**(-3))
    
#     # If y<<1   (Approx_Det = True ), we use delta func approx for Doppler,
#     # Otherwise (Approx_Det = False), we do numerical integral for Doppler
#     DeltaI_over_eps2_Doppler  = np.where(Approx_Det
#                           ,DeltaI_over_eps2_Doppler_delta(x,m_Aprime,T0,units=units)
#                           ,DeltaI_over_eps2_Doppler_int(x,x_prime_int,m_Aprime,T0,units=units))
    
#     return DeltaI_over_eps2_Doppler
    

# # DeltaI/eps^2 (Total)
# # Full Integration
# def DeltaI_over_eps2_y_int(x, x_prime_int, m_Aprime, T0, units = 'eV_per_cmSq'):
    
#     # Delta_I(A->Ap)
#     # int over xp
#     # 1D array: N_x
#     DeltaI_over_eps2_y_int = DeltaI_over_eps2_Y_int(x,x_prime_int,m_Aprime,T0,units=units) + DeltaI_over_eps2_Doppler_int(x,x_prime_int,m_Aprime,T0,units=units)
    
#     return DeltaI_over_eps2_y_int


# # Delta_I/eps^2 (Total): 
# # Y-contribution(int) + Doppler-contribution(switch)
# def DeltaI_over_eps2_y_switch(x, x_prime_int, m_Aprime, T0, units = 'eV_per_cmSq'):
    
#     DeltaI_over_eps2  = DeltaI_over_eps2_Y_int(x,x_prime_int,m_Aprime,T0,units=units) + DeltaI_over_eps2_Doppler_switch(x,x_prime_int,m_Aprime,T0,units=units)

#     return DeltaI_over_eps2


# # # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# # # Test of the Code
# # # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# # # x_test = np.array( [1, 10, 30] )
# # x_test = np.array( [0.1,1,15] )

# # # x_prime_int_test = np.logspace(-3,10,300000)
# # x_prime_int_test = np.logspace(-3,2,100000)

# # # x_prime_int_test_smallgrid = np.logspace(-3,10,300)
# # x_prime_int_test_smallgrid = np.logspace(-2,2,300)

# # # m_Ap_test = 10**(-9)    #in eV
# # # m_Ap_test = 10**(-8.5)    #in eV
# # m_Ap_test = 10**(-8)    #in eV
# # # m_Ap_test = 10**(-7)    #in eV
# # # m_Ap_test = 10**(-6)    #in eV
# # # m_Ap_test = np.sqrt( mAsq(10**4, 0) )

# # zres_test = zres_interp(m_Ap_test, 0)[0]  # Here I choose x_prime = 0

# # T0_test   = TCMB_0    #in eV

# # # DeltaI in y-era: Total, Integration
# # DeltaI_over_eps2_y_int_test          = DeltaI_over_eps2_y_int( x_test, x_prime_int_test, m_Ap_test, T0_test, units = 'MJy')

# # # DeltaI in y-era: Y-part
# # DeltaI_over_eps2_Y_int_test          = DeltaI_over_eps2_Y_int( x_test, x_prime_int_test, m_Ap_test, T0_test, units = 'MJy')

# # # DeltaI in y-era: Doppler-part
# # DeltaI_over_eps2_Doppler_int_test    = DeltaI_over_eps2_Doppler_int( x_test, x_prime_int_test, m_Ap_test, T0_test, units = 'MJy')

# # # DeltaI in y-era: Doppler-part (y_gamma<<1)
# # DeltaI_over_eps2_Doppler_delta_test  = DeltaI_over_eps2_Doppler_delta( x_test, m_Ap_test, T0_test, units = 'MJy')

# # # DeltaI in y-era: Doppler-part with switch
# # DeltaI_over_eps2_Doppler_switch_test = DeltaI_over_eps2_Doppler_switch( x_test, x_prime_int_test, m_Ap_test, T0_test, units = 'MJy')

# # # DeltaI in y-era: Total with switch
# # DeltaI_over_eps2_y_switch_test       = DeltaI_over_eps2_y_switch( x_test, x_prime_int_test_smallgrid, m_Ap_test, T0_test, units = 'MJy')

# # print('--------------------------------------------------')
# # print('For (int)')
# # print('--------------------------------------------------')
# # print('Lower Int Limit of x_prime = ', x_prime_int_test[0])
# # print('Upper Int Limit of x_prime = ', x_prime_int_test[-1])
# # print('Grid Number of x_prime     = ', len(x_prime_int_test))
# # print('')
# # print('--------------------------------------------------')
# # print('For (switch)')
# # print('--------------------------------------------------')
# # print('Lower Int Limit of x_prime = ', x_prime_int_test_smallgrid[0])
# # print('Upper Int Limit of x_prime = ', x_prime_int_test_smallgrid[-1])
# # print('Grid Number of x_prime     = ', len(x_prime_int_test_smallgrid))
# # print('============================================================================')
# # print('x                                                         = ', x_test)
# # print('T0                                                        = ', T0_test/K, 'K')
# # print('m_Ap                                                      = ', m_Ap_test, 'eV')
# # print('z_res                                                     = ', zres_test)
# # print('y_gamma                                                   = ', y_gamma(zres_test) )
# # print('============================================================================')
# # print('Delta_I_Doppler(delta)/eps^2                              = ', DeltaI_over_eps2_Doppler_delta_test, 'MJy')
# # print('Delta_I_Doppler(int)/eps^2                                = ', DeltaI_over_eps2_Doppler_int_test, 'MJy')
# # print('Delta_I_Doppler(switch)/eps^2                             = ', DeltaI_over_eps2_Doppler_switch_test, 'MJy')
# # print('Delta_I_Doppler(delta)/Delta_I_Doppler(int)               = ', DeltaI_over_eps2_Doppler_delta_test/DeltaI_over_eps2_Doppler_int_test )
# # print('============================================================================')
# # print('Delta_I(int)/eps^2                                        = ', DeltaI_over_eps2_y_int_test, 'MJy')
# # print('')
# # print('Delta_I_Y(int)/Delta_I_Doppler(int)                       = ', DeltaI_over_eps2_Y_int_test/DeltaI_over_eps2_Doppler_int_test )
# # print('(Delta_I_Y(int) + Delta_I_Doppler(int) )/Delta_I_y(int)   = ', (DeltaI_over_eps2_Y_int_test+DeltaI_over_eps2_Doppler_int_test)/DeltaI_over_eps2_y_int_test )
# # print('============================================================================')
# # print('Delta_I(switch)/Delta_I(int)                              = ', DeltaI_over_eps2_y_switch_test/DeltaI_over_eps2_y_int_test )
# # print('============================================================================')

## 5C. Spectral Distortion in $\mu$-y Transition Era

The spectral distortion in $\mu-y$ transition era is
\begin{equation}
\Delta I(x; m_{A'}, \epsilon;  \color{magenta}{T_0}) = - \frac{1}{2 \zeta(3)} \int dx' G^{(Chluba)}(x,x',z_\text{res}; \color{magenta}{T_0}) \frac{x'^2}{e^{x'}-1} P_{\gamma \rightarrow A'}(x',z_\text{res}),
\end{equation}

where 

\begin{equation}
G^{(Chluba)}(x,x',z_\text{res};\color{magenta}{T_0}) = \mathscr{T}_\mu(z_\text{res}) \times G^{(Chluba)}_\mu(x,x',z_\text{res};\color{magenta}{T_0}) + \mathscr{T}_y(z_\text{res}) \times G^{(Chluba)}_y(x,x',z_\text{res};\color{magenta}{T_0}).
\end{equation}

Here, the transition functions are

\begin{equation}
\mathscr{T}_\mu(z) = 1 - \exp\left[ - \left(\frac{1+z}{5.8 \times 10^4}\right)^{1.88} \right]  \quad \text{and} \quad \mathscr{T}_y(z) = \frac{1}{1+\left(\frac{1+z}{6 \times 10^4}\right)^{2.58}}.
\end{equation}

Therefore, we have

\begin{equation}
\Delta I(x; m_{A'}, \epsilon;  \color{magenta}{T_0}) = \mathscr{T}_\mu(z_\text{res}) \times \Delta I_\mu(x; m_{A'}, \epsilon;  \color{magenta}{T_0}) + \mathscr{T}_y(z_\text{res}) \times \Delta I_y(x; m_{A'}, \epsilon;  \color{magenta}{T_0})
\end{equation}


The CMB spectrum after distortion is
\begin{equation}
I^\text{(dist)}(x, m_{A'}, \epsilon; \color{magenta}{T_0}) = I^{(0)}(x;  \color{magenta}{T_0}) + \Delta I(x, m_{A'}, \epsilon; \color{magenta}{T_0}).
\end{equation}

## Reasonable transition functions

These transition functions are found by Xucheng after comparing on the shape of Green's function with Chluba15.

### Green's function in transition era

\begin{equation}
G = \mathcal{T}_\mu \times G_M + (1-\mathcal{T}_\mu) \times (G_Y + G_\mathrm{Doppler})
\end{equation}

### Exponential Transition Function (General)

\begin{equation}
\mathscr{T}_\mu(z) = 1 - \exp\left[ - \left(\frac{1+z}{1+z_\mathrm{trans}}\right)^{\mathrm{Power}} \right].
\end{equation}

### Transition Function (Chluba 2015):

\begin{equation}
z_\mathrm{trans} = 5.8 \times 10^4, \quad \mathrm{Power} = 1.88
\end{equation}

### Transition Function (Xucheng):

\begin{equation}
z_\mathrm{trans} = (5.8 \times 10^4, 10^5, 1.4 \times 10^5, 1.7 \times 10^5), \quad \mathrm{Power} = (1.88, 3, 5)
\end{equation}

### Best Value (Xucheng)

\begin{equation}
z_\mathrm{trans} = 1.7 \times 10^5, \quad \mathrm{Power} = (3,5)
\end{equation}


In [ ]:
# # Transition Function (Xucheng):
# def Ttrans_mu_XG_5p8e4_1p88(z):
    
#     z_trans       = 5.8 * 10**4
#     Power_trans   = 1.88
    
#     Ttrans_mu = 1 - np.exp( -((1+z)/(1+z_trans))**Power_trans )
    
#     return Ttrans_mu



# def Ttrans_mu(z):
    
#     #Ttrans_mu = Ttrans_mu_Chluba15(z)        # (Chluba2015)
    
#     Ttrans_mu = Ttrans_mu_XG_5p8e4_1p88(z)
    
    
#     # Ttrans_mu = Ttrans_mu_XG_5p8e4_3(z)
#     # Ttrans_mu = Ttrans_mu_XG_5p8e4_5(z)
    
#     # Ttrans_mu = Ttrans_mu_XG_10e4_1p88(z)
#     # Ttrans_mu = Ttrans_mu_XG_10e4_3(z)
#     # Ttrans_mu = Ttrans_mu_XG_10e4_5(z)
    
#     # Ttrans_mu = Ttrans_mu_XG_14e4_1p88(z)
#     # Ttrans_mu = Ttrans_mu_XG_14e4_3(z)
#     # Ttrans_mu = Ttrans_mu_XG_14e4_5(z)        # (Good)
    
#     # Ttrans_mu = Ttrans_mu_XG_17e4_1p88(z)
#     # Ttrans_mu = Ttrans_mu_XG_17e4_3(z)        # (2nd Best)
#     # Ttrans_mu = Ttrans_mu_XG_17e4_5(z)        # (Best)
    
#     # Ttrans_mu = Ttrans_mu_XG_19e4_1p88(z)
#     # Ttrans_mu = Ttrans_mu_XG_19e4_3(z)
#     # Ttrans_mu = Ttrans_mu_XG_19e4_5(z)
    
#     return Ttrans_mu


# def Ttrans_y(z):
    
#     Ttrans_y = 1/( 1 + ((1+z)/6e4)**2.58 )
    
#     return Ttrans_y

In [ ]:
# z_test = np.logspace(2,7,100)

# Reg_test=10**(-5)

# plt.figure()

# set_matplotlib_formats('retina')
# plt.figure(figsize=(9,6))

# plt.xscale('log')
# # plt.yscale('log')

# color_5p8e4_1p88 = lighten_color('gray', 0.3)
# color_5p8e4_3    = lighten_color('gray', 1)
# color_5p8e4_5    = lighten_color('gray', 2)

# color_10e4_1p88  = lighten_color('red', 0.3)
# color_10e4_3     = lighten_color('red', 1)
# color_10e4_5     = lighten_color('red', 1.5)

# color_14e4_1p88  = lighten_color('green', 0.3)
# color_14e4_3     = lighten_color('green', 1)
# color_14e4_5     = lighten_color('green', 1.2)

# color_17e4_1p88  = lighten_color('blue', 0.3)
# color_17e4_3     = lighten_color('blue', 1)
# color_17e4_5     = lighten_color('blue', 1.2)


# plt.plot(z_test,  Ttrans_mu_Chluba15(z_test)     , label = r'$\mathcal{T}_\mu$: Chluba 15, $z_\mathrm{trans} = 5.8 \times 10^4$, $\mathrm{Power} = 1.88$', color = color_5p8e4_1p88  )
# plt.plot(z_test,  Ttrans_mu_XG_5p8e4_3(z_test)   , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 5.8 \times 10^4$, $\mathrm{Power} = 3$',           color = color_5p8e4_3 )
# plt.plot(z_test,  Ttrans_mu_XG_5p8e4_5(z_test)   , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 5.8 \times 10^4$, $\mathrm{Power} = 5$',           color = color_5p8e4_5 )

# plt.plot(z_test,  Ttrans_mu_XG_10e4_1p88(z_test) , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 10^5$, $\mathrm{Power} = 1.88$',                   color = color_10e4_1p88  )
# plt.plot(z_test,  Ttrans_mu_XG_10e4_3(z_test)    , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 10^5$, $\mathrm{Power} = 3$',                      color = color_10e4_3     )
# plt.plot(z_test,  Ttrans_mu_XG_10e4_5(z_test)    , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 10^5$, $\mathrm{Power} = 5$',                      color = color_10e4_5     )

# plt.plot(z_test,  Ttrans_mu_XG_14e4_1p88(z_test) , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.4 \times 10^5$, $\mathrm{Power} = 1.88$',        color = color_14e4_1p88  )
# plt.plot(z_test,  Ttrans_mu_XG_14e4_3(z_test)    , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.4 \times 10^5$, $\mathrm{Power} = 3$',           color = color_14e4_3     )
# plt.plot(z_test,  Ttrans_mu_XG_14e4_5(z_test)    , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.4 \times 10^5$, $\mathrm{Power} = 5$',           color = color_14e4_5     )

# plt.plot(z_test,  Ttrans_mu_XG_17e4_1p88(z_test) , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.7 \times 10^5$, $\mathrm{Power} = 1.88$',        color = color_17e4_1p88  )
# plt.plot(z_test,  Ttrans_mu_XG_17e4_3(z_test)    , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.7 \times 10^5$, $\mathrm{Power} = 3$',           color = color_17e4_3     )
# plt.plot(z_test,  Ttrans_mu_XG_17e4_5(z_test)    , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.7 \times 10^5$, $\mathrm{Power} = 5$',           color = color_17e4_5     )

# # plt.plot(z_test,  Ttrans_mu_XG_19e4_1p88(z_test) , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.9 \times 10^5$, $\mathrm{Power} = 1.88$', color = lighten_color('purple', 0.3))
# # plt.plot(z_test,  Ttrans_mu_XG_19e4_3(z_test)    , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.9 \times 10^5$, $\mathrm{Power} = 3$', color = lighten_color('purple', 1) )
# # plt.plot(z_test,  Ttrans_mu_XG_19e4_5(z_test)    , label = r'$\mathcal{T}_\mu$: XG, $z_\mathrm{trans} = 1.9 \times 10^5$, $\mathrm{Power} = 5$', color = lighten_color('purple', 1.2) )

# plt.plot(z_test,  Ttrans_mu(z_test), label = r'$\mathcal{T}_\mu$: Chosen in our code', color = lighten_color('cyan', 1.25), linewidth=6)

# # plt.title(r'$G_\mu(x,x^\prime,z^\prime)$ Normalization')

# plt.xlabel(r'$z$')
# plt.ylabel(r'Transition Function')

# plt.ylim(0, 1.2)

# plt.legend(fontsize=11)

# plt.savefig('Trans_Func.pdf')

In [ ]:


# def DeltaI_over_eps2_muy_trans(x, x_prime_int, m_Aprime, T0, units = 'eV_per_cmSq'):
    
#     # Here I choose x_prime = 0
#     z_res = zres_interp(m_Aprime, 0)[0]
    
#     # Check whether in the pure-mu region
#     mu_era_Check = (z_res > z_trans_2)
    
#     # DeltaI/eps^2(total, in mu/y transition era)
# #     DeltaI_over_eps2_muy_trans =  np.where(mu_era_Check,
# #                                     DeltaI_over_eps2_mu_M_int(x, x_prime_int, m_Aprime, T0, units = units),
# #                                     ( Ttrans_mu(z_res) * DeltaI_over_eps2_mu_M_int(x, x_prime_int, m_Aprime, T0, units = units) 
# #                                     + Ttrans_y(z_res)  * DeltaI_over_eps2_y_switch(x, x_prime_int, m_Aprime, T0, units = units) ) 
# #                                           )
    
#     DeltaI_over_eps2_muy_trans =  np.where(mu_era_Check,
#                                         DeltaI_over_eps2_mu_M_int(x, x_prime_int, m_Aprime, T0, units = units),
#                                         ( Ttrans_mu(z_res) * DeltaI_over_eps2_mu_M_int(x, x_prime_int, m_Aprime, T0, units = units) 
#                                         + (1-Ttrans_mu(z_res)) * DeltaI_over_eps2_y_switch(x, x_prime_int, m_Aprime, T0, units = units) ) 
#                                               )

#     # DeltaI_over_eps2_muy_trans =  Ttrans_mu(z_res) * DeltaI_over_eps2_mu_M_int(x, x_prime_int, m_Aprime, T0, units = units) 
#     # DeltaI_over_eps2_muy_trans =  Ttrans_y(z_res) * DeltaI_over_eps2_y_switch(x, x_prime_int, m_Aprime, T0, units = units)
#     # DeltaI_over_eps2_muy_trans =  DeltaI_over_eps2_y_switch(x, x_prime_int, m_Aprime, T0, units = units)
    
#     return DeltaI_over_eps2_muy_trans

# # [[[This is what we used in chi^2 analysis!!!]]]
# def I0_dist_muy_trans(x, x_prime_int, m_Aprime, eps, T0, units = 'eV_per_cmSq'):
    
#     # CMB intensity after distortion
#     # 1D array: N_x
#     I0_dist_y = I0(x, T0, units=units) + eps**2 * DeltaI_over_eps2_muy_trans(x, x_prime_int, m_Aprime, T0, units = units)
    
#     return I0_dist_y



# # # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# # # Test of the Code
# # # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# # x_test = np.array( [0.1,1,15] )

# # # x_prime_int_test = np.logspace(-2,10,100000)
# # # x_prime_int_test = np.logspace(-3,2,100000)
# # x_prime_int_test = np.logspace(-2,2,300)

# # m_Ap_test = 10**(-7)    #in eV

# # zres_test = zres_interp(m_Ap_test, 0)[0]  # Here I choose x_prime = 0

# # T0_test   = TCMB_0    #in eV

# # eps_test = 10**(-7)

# # print('============================================================================')
# # print('x                                                         = ', x_test)
# # print('T0                                                        = ', T0_test/K, 'K')
# # print('m_Ap                                                      = ', m_Ap_test, 'eV')
# # print('z_res                                                     = ', zres_test)
# # print('y_gamma                                                   = ', y_gamma(zres_test) )
# # print('============================================================================')